Prepare the Bunnies' Escape
===========================

You're awfully close to destroying the LAMBCHOP doomsday device and freeing Commander Lambda's bunny prisoners, but once they're free of the prison blocks, the bunnies are going to need to escape Lambda's space station via the escape pods as quickly as possible. Unfortunately, the halls of the space station are a maze of corridors and dead ends that will be a deathtrap for the escaping bunnies. Fortunately, Commander Lambda has put you in charge of a remodeling project that will give you the opportunity to make things a little easier for the bunnies. Unfortunately (again), you can't just remove all obstacles between the bunnies and the escape pods - at most you can remove one wall per escape pod path, both to maintain structural integrity of the station and to avoid arousing Commander Lambda's suspicions. 

You have maps of parts of the space station, each starting at a prison exit and ending at the door to an escape pod. The map is represented as a matrix of 0s and 1s, where 0s are passable space and 1s are impassable walls. The door out of the prison is at the top left (0,0) and the door into an escape pod is at the bottom right (w-1,h-1). 

Write a function solution(map) that generates the length of the shortest path from the prison door to the escape pod, where you are allowed to remove one wall as part of your remodeling plans. The path length is the total number of nodes you pass through, counting both the entrance and exit nodes. The starting and ending positions are always passable (0). The map will always be solvable, though you may or may not need to remove a wall. The height and width of the map can be from 2 to 20. Moves can only be made in cardinal directions; no diagonal moves are allowed.

Languages
=========

To provide a Python solution, edit solution.py
To provide a Java solution, edit Solution.java

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Python cases --
Input:
solution.solution([[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]])
Output:
    7

Input:
solution.solution([[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]])
Output:
    11

In [2]:
############# EXAUSTIVE SEARCH SOLUTION ##################

# References
# [1] - https://www.python.org/doc/essays/graphs/
# [2] - https://en.wikipedia.org/wiki/Taxicab_geometry
################

# \cite{1}
class Graph(object): 
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict

    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.__graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        #edge = set(edge)
        #(vertex1, vertex2) = tuple(edge)
        if edge[0] in self.__graph_dict:
            self.__graph_dict[edge[0]].append(edge[1])
        else:
            self.__graph_dict[edge[0]] = [edge[1]]

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            for neighbour in self.__graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges
    def get_dict(self):
        return self.__graph_dict
    
    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res
        
    def find_path(self, start_vertex, end_vertex, path=None):
        """ find a path from start_vertex to end_vertex 
            in graph """
        if path == None:
            path = []
        graph = self.__graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return path
        if start_vertex not in graph:
            return None
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_path = self.find_path(vertex, 
                                               end_vertex, 
                                               path)
                if extended_path: 
                    return extended_path
        return None

    def find_shortest_path(self, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if not start in self.__graph_dict:
            return None
        shortest = None
        for node in self.__graph_dict[start]:
            if node not in path:
                newpath = self.find_shortest_path(node, end, path)
                if newpath:
                    if not shortest or len(newpath) < len(shortest):
                        shortest = newpath
        return shortest

#########
class Map:
    def __init__(self, mapMatrix):
        self.h = len(mapMatrix)
        self.w = len(mapMatrix[0])        
        self.mapMatrix = mapMatrix
        
    def buildGraph (self):
        self.mapGraph = Graph()
        for i in range(self.h): # i=lines counter
            for j in range(self.w): # j = columns counter
                if self.mapMatrix[i][j]==0: 
                    n = self.nodeMakeName(i,j)
                    self.mapGraph.add_vertex(n)
                    #print("adding node (", n, ")")
                    for v in self.mapGraph.vertices():
                        if self.areNodesNeighbors (n, v):
                            self.mapGraph.add_edge([n,v])
                            self.mapGraph.add_edge([v,n])
                            #print ("adding edge (", n ,'<>', v, ')')
                        
    def nodeMakeName(self, i, j):
        return "N" + str(i).zfill(2) + str(j).zfill(2)
    
    def areNodesNeighbors (self, n, v):
        manhatanDistance = abs(int(n[1:3]) - int(v[1:3])) # \cite{2}
        manhatanDistance += abs(int(n[3:5]) - int(v[3:5]))
        if manhatanDistance==1: return True
        else: return False
    
    def scape(self):
        return self.mapGraph.find_shortest_path (self.nodeMakeName(self.h-1, self.w-1), self.nodeMakeName(0,0))

##############    
def solution (map):
    h = len(map)
    w = len(map[0])  
    mapOriginal = Map (map)
    mapOriginal.buildGraph()

    scapes = []
    scapes.append(len(mapOriginal.scape()))
    for i in range(h): # i=lines counter
        for j in range(w): # j = columns counter
            if map[i][j]==1:
                map[i][j]=0
                mapNew = Map(map)
                mapNew.buildGraph()
                scapes.append(len(mapNew.scape()))
                map[i][j]=1
    return min(scapes)

11

In [12]:
# References
# https://codereview.stackexchange.com/a/152188
# https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/

def bfsMaze(maze, sourceX, sourceY):
    w = len(maze[0])
    h = len(maze)
    mazeDist = [[None for i in range(w)] for i in range(h)] # mark all nodes as unvisited
    mazeDist[sourceX][sourceY] = 1 # Distances begin in 1

    queue = [(sourceX, sourceY)]
    while queue:
        x, y = queue.pop(0)
        for direction in [[1,0],[-1,0],[0,-1],[0,1]]: # for all adjacent nodes
            x2, y2 = x + direction[0], y + direction[1]
            if 0 <= x2 < h and 0 <= y2 < w:
                if mazeDist[x2][y2] is None: # if Unvisited
                    mazeDist[x2][y2] = mazeDist[x][y] + 1
                    if maze[x2][y2] == 1: continue
                    queue.append((x2, y2)) 
                  
    return mazeDist

def solution(maze):
    w = len(maze[0])
    h = len(maze)
    bfs1 = bfsMaze(maze, 0, 0)
    bfs2 = bfsMaze(maze, h-1, w-1)

    shortest = 2 ** 32-1
    for i in range(h):
        for j in range(w):
            if bfs1[i][j] and bfs2[i][j]:
                shortest = min(bfs1[i][j] + bfs2[i][j] - 1, shortest)
    return shortest

11

In [ ]:
def printMaze(M):
    print ('-'*(len(M[0])*3 + 1))
    for row in M:
        print ('|', end ="")
        for m in row:
            print (str(m).zfill(2), end =" ")
        print ('|')
    print ('-'*(len(M[0])*3 + 1))

In [106]:
print ('-'*(len(maze[0])*2 + 2))
for i in range(h):
    print ('|', end ="")
    for j in range(w):
        n = maze[i][j]
        if n==1: print('██', end='')
        elif (i,j) in path: print ('..', end='')
        else: print ("  ", end='')
    print ('|')
print ('-'*(len(maze[0])*2 + 2))

--------------
|..........  |
|██████████..|
|  ........  |
|..██████████|
|..██████████|
|  ..........|
--------------
